In [13]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize

In [14]:
df=pd.read_csv("tweets.csv",index_col=0)
df=df.drop(columns=["id","retweet","link"])
df

,date,text,author
0,Oct 7,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump
1,Oct 10,Is this really America? Terrible!pic.twitter.c...,DonaldTrump
2,Oct 8,The media and establishment want me out of the...,DonaldTrump
3,Oct 8,Certainly has been an interesting 24 hours!,DonaldTrump
4,Oct 10,Debate polls look great - thank you!\n#MAGA #A...,DonaldTrump
...,...,...,...
17211,12 May 2009,"""My persona will never be that of a wallflower...",DonaldTrump
17212,8 May 2009,New Blog Post: Celebrity Apprentice Finale and...,DonaldTrump
17213,8 May 2009,Donald Trump reads Top Ten Financial Tips on L...,DonaldTrump
17214,4 May 2009,Donald Trump will be appearing on The View tom...,DonaldTrump


In [15]:
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

def traitement(phrase):
    phrase=phrase.lower()
    texte2=(str(phrase).replace(',',' ')).lower()
    texte2=texte2.replace("'",' ')
    texte2=texte2.replace("-",' ')
    texte2=texte2.replace(".",' ')
    texte2=texte2.replace("*",' ')
    texte2=texte2.replace("\\",' ')
    texte2=texte2.replace("\n",' ')
    texte2=texte2.replace("/",' ')
    texte2=texte2.replace("!",' ')
    texte2=texte2.replace("?",' ')
    texte2=texte2.replace("\"",' ')
    texte2=texte2.replace("'",' ')
    texte2=texte2.replace("\#",' ')
    texte2=texte2.split()
    texte2=[token for token in texte2 if len(token) and token.lower() not in stopwords]
    return ' '.join(texte2)


In [16]:
df['textmodif']=df['text'].map(traitement)
df

,date,text,author,textmodif
0,Oct 7,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump,statement pic twitter com wazigoqqmq
1,Oct 10,Is this really America? Terrible!pic.twitter.c...,DonaldTrump,really america terrible pic twitter com wiwc61...
2,Oct 8,The media and establishment want me out of the...,DonaldTrump,media establishment want race badly never drop...
3,Oct 8,Certainly has been an interesting 24 hours!,DonaldTrump,certainly interesting 24 hours
4,Oct 10,Debate polls look great - thank you!\n#MAGA #A...,DonaldTrump,debate polls look great thank #maga #americafi...
...,...,...,...,...
17211,12 May 2009,"""My persona will never be that of a wallflower...",DonaldTrump,persona never wallflower i’d rather build wall...
17212,8 May 2009,New Blog Post: Celebrity Apprentice Finale and...,DonaldTrump,new blog post: celebrity apprentice finale les...
17213,8 May 2009,Donald Trump reads Top Ten Financial Tips on L...,DonaldTrump,donald trump reads top ten financial tips late...
17214,4 May 2009,Donald Trump will be appearing on The View tom...,DonaldTrump,donald trump appearing view tomorrow morning d...


In [17]:
s = 300
alpha = 0.2


t_data = [TaggedDocument(words = word_tokenize(_d.lower()),tags = [str(i)]) for i,_d in enumerate(df['textmodif'])]
model = Doc2Vec(t_data,
                       vector_size = s,
                      alpha = alpha,
                      min_alpha = 0.0002,
                      min_count = 3,
                      dm = 1)
model.save('tweetmodel.model')

In [18]:
model=Doc2Vec.load('tweetmodel.model')
model.wv.get_vector('maga')

array([-2.36331031e-01, -2.15514719e-01, -2.10673824e-01,  5.79002500e-01,
        2.11669490e-01,  3.92164811e-02, -1.36076733e-01,  2.46467933e-01,
        1.24396130e-01,  6.72970945e-03,  7.01923519e-02,  1.10626221e-01,
        2.72128940e-01, -2.78545171e-01,  2.10533276e-01, -2.69423962e-01,
       -4.64876860e-01,  1.39546394e-01, -1.20241694e-01, -1.49096325e-01,
       -3.17276925e-01, -2.70045668e-01, -2.53245711e-01, -1.17039345e-01,
       -8.38885963e-01,  4.11128066e-02,  3.33693832e-01, -5.35140574e-01,
        5.36312908e-02, -3.18649858e-01, -2.97595024e-01, -2.14520097e-01,
        5.60100853e-01, -2.41550788e-01,  1.01733521e-01, -8.12484920e-02,
        2.82992542e-01,  4.16713022e-02,  3.02343238e-02,  3.49511415e-01,
        2.84633845e-01,  3.07101011e-01, -2.88353320e-02,  6.64498806e-02,
       -7.65047908e-01,  9.72774848e-02,  6.02779567e-01,  1.41098231e-01,
       -5.26964843e-01,  7.77666271e-02, -2.93765515e-01, -2.58742332e-01,
       -6.21592700e-01,  

In [19]:
len(model.wv.get_vector('one'))

300

In [20]:
def norme(vec):
    return np.sqrt(np.sum(vec*vec))
def prediction(vec1,vec2):
    val=np.sum((vec1*vec2))
    val/=(norme(vec1)*norme(vec2))
    if val>1:
        return np.arccos(1)#because of the structure of floats it can appen that the result is almost 1 but superior
    else:
        return np.arccos(val)
        
def taille(vec1,vec2):
    return abs(norme(vec1)-norme(vec2))

In [37]:
def vectorization(word):
    T=word.split(' ')
    vector=sum([vec(i) for i in T])
    return vector

def vec(mot):
    try:
        x=model.wv.get_vector(mot)
        return x
    except:
        return np.array([0]*s,dtype=np.float32)

def similarity(word1,word2):
    w1,w2=traitement(word1),traitement(word2)
    v1,v2=vectorization(w1),vectorization(w2)
    #print(v1,v2)
    return prediction(v1,v2)

In [38]:
similarity("France s",df['text'][10000])

1.4518285

In [39]:
def get_20_best(phrase):
    res={tweet:similarity(phrase,tweet) for tweet in df['text']}
    res=sorted(res, key= lambda A: res[A])
    return res[:20]

In [40]:
get_20_best("Build a wall !!!")

c:\users\minim\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in float_scalars
  """
c:\users\minim\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in greater
  


['Mexico will pay for the wall!',
 "Obama says a WALL at our southern border won't enhance our security (wrong) and yet he now wants to build a much bigger wall (fence) at W.H.",
 'When will the Democrats, and Hillary in particular, say "we must build a wall, a great wall, and Mexico is going to pay for it?" Never!',
 '"@Retrogirl01: @AP Mr. Trump is right! BUILD THE WALL! @realDonaldTrump"',
 'Hillary will never reform Wall Street. She is owned by Wall Street!',
 'Jeb Bush just talked about my border proposal to build a "fence." It\'s not a fence, Jeb, it\'s a WALL, and there\'s a BIG difference!',
 '.@AnnCoulter has been amazing. We will win and establish strong borders, we will build a WALL and Mexico will pay. We will be great again!',
 "BREAKING - Border security rally in Phoenix, AZ at 2PM MST has been moved to @PhoenixConvCtr! Build a wall! Let's Make America Great Again!",
 'We will stop heroin and other drugs from coming into New Hampshire from our open southern border. We wil